In [1]:
import requests

Api_key = "bbuUFrmvBachgADcHjZRxT8Bq2G4j46E8dFbWNgI"
url = f"https://api.nasa.gov/neo/rest/v1/feed?start_date=2025-01-01&end_date=2025-01-07&api_key={Api_key}"

response = requests.get(url)
data = response.json() #json format data as response

In [5]:
data.keys()

dict_keys(['links', 'element_count', 'near_earth_objects'])

In [40]:
data['near_earth_objects']

{'2025-01-07': [{'links': {'self': 'http://api.nasa.gov/neo/rest/v1/neo/2226514?api_key=bbuUFrmvBachgADcHjZRxT8Bq2G4j46E8dFbWNgI'},
   'id': '2226514',
   'neo_reference_id': '2226514',
   'name': '226514 (2003 UX34)',
   'nasa_jpl_url': 'https://ssd.jpl.nasa.gov/tools/sbdb_lookup.html#/?sstr=2226514',
   'absolute_magnitude_h': 20.14,
   'estimated_diameter': {'kilometers': {'estimated_diameter_min': 0.2492039814,
     'estimated_diameter_max': 0.5572370428},
    'meters': {'estimated_diameter_min': 249.2039814409,
     'estimated_diameter_max': 557.2370427654},
    'miles': {'estimated_diameter_min': 0.1548481272,
     'estimated_diameter_max': 0.3462509385},
    'feet': {'estimated_diameter_min': 817.5983904706,
     'estimated_diameter_max': 1828.2055793866}},
   'is_potentially_hazardous_asteroid': True,
   'close_approach_data': [{'close_approach_date': '2025-01-07',
     'close_approach_date_full': '2025-Jan-07 00:08',
     'epoch_date_close_approach': 1736208480000,
     'relat

In [120]:
asteroids_data = []
target = 10000
url = f"https://api.nasa.gov/neo/rest/v1/feed?start_date=2025-01-01&end_date=2025-01-07&api_key={Api_key}"
tab1 = []
tab2 = []
while len(asteroids_data) < target:
    response = requests.get(url)
    data = response.json()
    details = data['near_earth_objects']
    for date,info in details.items():
      for i in info:
        asteroids_data.append(dict(id = i['id'], #neo_reference_id same as id
        name = i['name'],
        mag= float(i['absolute_magnitude_h']),
        dia_min = float(i['estimated_diameter']['kilometers']['estimated_diameter_min']),
        dia_max = float(i['estimated_diameter']['kilometers']['estimated_diameter_max']),
        hazard = bool(i['is_potentially_hazardous_asteroid']),
        date =  (i['close_approach_data'][0]['close_approach_date']),
        velocity = float(i['close_approach_data'][0]['relative_velocity']['kilometers_per_hour']),
        astronomical = float(i['close_approach_data'][0]['miss_distance']['astronomical']),
        missed_dist_kilo = float(i['close_approach_data'][0]['miss_distance']['kilometers']),                   
        lunar_dist =  float(i['close_approach_data'][0]['miss_distance']['lunar']),
        orbiting_body = (i['close_approach_data'][0]['orbiting_body'])
        ))

        tab1.append((
         i['id'], #neo_reference_id same as id
         i['name'],
         float(i['absolute_magnitude_h']),
         float(i['estimated_diameter']['kilometers']['estimated_diameter_min']),
         float(i['estimated_diameter']['kilometers']['estimated_diameter_max']),
         bool(i['is_potentially_hazardous_asteroid'])
         
        ))

        tab2.append((
         i['id'], #neo_reference_id same as id
         (i['close_approach_data'][0]['close_approach_date']),
         float(i['close_approach_data'][0]['relative_velocity']['kilometers_per_hour']),
         float(i['close_approach_data'][0]['miss_distance']['astronomical']),
         float(i['close_approach_data'][0]['miss_distance']['kilometers']),                   
         float(i['close_approach_data'][0]['miss_distance']['lunar']),
         (i['close_approach_data'][0]['orbiting_body'])
        ))
          
        if len(asteroids_data) >= target:
            break
      if len(asteroids_data) >= target:
          break

url = data['links']['next']

In [122]:
len(asteroids_data)

10000

In [61]:
!pip install pymysql

In [124]:
import pymysql

In [126]:
connection = pymysql.connect(

                            host = "localhost", # IP address of your server
                            user = "rahila",
                            password = "12345",
                            database = "asteriod"
                           
                    )
cursor = connection.cursor()

In [128]:
connection

In [130]:
cursor.execute("""create table asteroids 
                (id INT, 
                name varchar(300), 
                absolute_magnitude_h FLOAT, 
                estimated_diameter_min_km FLOAT, 
                estimated_diameter_max_km FLOAT,
                is_potentially_hazardous_asteroid BOOL);""")

0

In [132]:
cursor.execute("""create table close_approach 
                (id INT, 
                close_approach_date DATE, 
                relative_velocity_kmph FLOAT, 
                astronomical FLOAT, 
                miss_distance_km FLOAT,
                miss_distance_lunar FLOAT,
                orbiting_body varchar(300));""")

0

In [134]:
insert = """
insert into asteroids
values
(%s, %s, %s, %s, %s, %s)
"""

cursor.executemany(insert,tab1)

connection.commit()

In [136]:
insert = """
insert into close_approach
values
(%s, %s, %s, %s, %s, %s, %s)
"""

cursor.executemany(insert,tab2)

connection.commit()